# Estimación de dirección de arribo de fuentes sonoras 

In [4]:
from tdoa_py import correlation, simulation_room

In [5]:
angle, wavs = simulation_room.sim_room_Nmics(
    wav_path = 'audios/anechoic/audio_anecoico_corto.wav',
    out_dir = 'audios/output/audio_corto',
    fs=48000,

    room_dim=(100, 100, 100),
    rt60=3,
    snr_db=70,

    n_mics=4,
    mic_d=0.1,
    mic_z=1.2,
    mic_directivity='omni',

    src_dist=50.0,
    src_az_deg=45.0,
    src_z=1.2
)

Fuente en: x=85.36, y=85.36, z=1.20


In [6]:

avg_angle_deg, avg_tdoa, hemi_avgs = correlation.estimate_doa_from_wavs(
    wav_folder = 'audios/output/audio_corto',
    mic_d = 0.1,
    fs = 48000
)


Mic 0: TDOA = 0.000417 s, Ángulo estimado = 44.39°
Mic 1: TDOA = 0.000208 s, Ángulo estimado = 44.39°
Mic 3: TDOA = -0.000208 s, Ángulo estimado = 224.39°

Hemisferio dominante: H1
H1: Promedio = 44.39°
H2: Promedio = 224.39°
